## Flask 
- 파이썬 코드를 사용하는 웹 프레임워크
- `https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world`
- install 
    - `pip3 install flask`

`pip3 install flask`

### nginx setting

nginx를 설치해야, 인터넷망으로 접속 가능 
- sudo apt-get update
- sudo apt-get install nginx
- sudo systemctl status nginx
- aws console에서 80번 포트 활성화
    - aws 보안그룹, `80` `0.0.0.0/0`
- 브라우져에 퍼블릭 ip입력해서 Welcome to Nginx 확인    

##### nginx setting file

- `sudo vi /etc/nginx/sites-available/default`

<pre>
server {
        listen 80 default_server;
        listen [::]:80 default_server;
</pre>

<pre>
server {
        listen 8080 default_server;
        listen [::]:8080 default_server;
</pre>

and 


맨 하단에 추가. 80번 포트로 리퀘스트 받으면, localhost:5000으로 서비스 해 주겠다는 뜻. 


to
<pre>
server {
    listen 80;
    location / {
        proxy_pass http://localhost:5000;
 }
}
</pre>




- 이렇게 하면, 8080으로 들어오면 아까 welcom to nginx가 뜨고, 80으로 접속하면 우리꺼가 뜨는 것. 


- 이제 nginx restart 
    - sudo systemctl restart nginx

#### Domain
- [freenom](https://www.freenom.com/en/index.html?lang=en)
- register한 후에, target에 내 IP 넣기만 하면 연결됨. 엄청 간단함. 

In [1]:
!mkdir -p hello/static
!mkdir -p hello/templates
!touch hello/hello.py
!touch hello/templates/index.html
!tree hello

hello
├── hello.py
├── static
└── templates
    └── index.html

2 directories, 2 files


In [44]:
!tree hello

hello
├── hello.py
├── index.html
├── model
│   └── clf.pkl
├── static
└── templates
    └── index.html

3 directories, 4 files


- hello.py : app 객체를 생성, route 설정
- static : js, css, imgage파일 등을 저장
- templates : html 코드를 저장

In [13]:
import os

# flask에서 알아서 __file__에 현재 실행되는 절대 경로를 넣어준다. 
path= os.path.dirname(os.path.realpath("file_example"))
print(os.path.realpath("file_example"))
path

/home/ubuntu/flask/file_example


'/home/ubuntu/flask'

In [46]:
%%writefile ./hello/hello.py
from flask import *
import os
import pickle
import numpy as np

app = Flask(__name__)




path= os.path.dirname(os.path.realpath(__file__))
with open("{}/model/clf.pkl".format(path), "rb") as f:
    model = pickle.load(f)

    

@app.route("/")
def hello():
    return "Hello World!"

@app.route("/user/<name>")
def user(name):
    return render_template('index.html', name=name)

@app.route('/api/data')
def api_data():
    data = {"noel":25, "mei": 25}
    return jsonify(data)

@app.route('/api/predic')
def predic():
    result = {"code": 200}  
    
    sentence = request.values.get("sentence")
    print(sentence)
    result['sentence'] = sentence
    result['category']= ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학"]
    result["result"] = np.round_(model.predict_proba([sentence])[0]*100, 2).tolist()
    
    print(result)
    print(type(result))
    
    return jsonify(result)

app.run(debug=True)

Overwriting ./hello/hello.py


In [50]:
%%writefile ./hello/templates/index.html
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Hello Flask</title>
</head>
<body>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.0/jquery.js" integrity="sha512-n/4gHW3atM3QqRcbCn6ewmpxcLAHGaDjpEBu4xZd47N0W2oQ+6q7oc3PXstrJYXcbNU1OHdQ1T7pAP+gi5Yu8g==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
  Hello {{ name }}
  <button class="result">Click!</button>
  <div class="data"></div>
  <input type="text" class="sentence">
  <button type="button" class="prediction-btn">Predict!!</button>
  <div class="prediction-wrap"></div>

  <script type="text/javascript">
    $(document).ready(function(){
      $('.result').on('click', function(){
        $.getJSON("/api/data", function(data){
            console.log(data);
            var tag = "<p> noel : "+ data.noel +"</p>";
                tag += "<p> mei : "+ data.mei +"</p>";
            $(".data").append(tag)
        })}
      )

      $('.prediction-btn').on('click', function(){
          var sentence = $('.sentence').val();
          var url ="/api/predic?sentence="+sentence;
          $.getJSON(url, function(data) {
                console.log(data)
                $('.prediction-wrap').empty()
                for(var i=0; i < data.category.length; i++){
                  var tag = "";
                    tag += "<p>"+data.category[i]+ " : ";
                    tag +=  data.result[i] +"%</p>";
                  $('.prediction-wrap').append(tag)
                }
          })
      })
    })
  </script>
</body>
</html>

Overwriting ./hello/templates/index.html


In [38]:
!python3 hello/hello.py

 * Serving Flask app "hello" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with fsevents reloader
 * Debugger is active!
 * Debugger PIN: 147-741-579
127.0.0.1 - - [20/Sep/2021 17:53:19] "GET /user/noel HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 17:53:20] "GET /api/data HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 18:58:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 18:59:01] "GET /users HTTP/1.1" 404 -
127.0.0.1 - - [20/Sep/2021 18:59:06] "GET /api/data HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 18:59:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2021 18:59:22] "GET /api/data HTTP/1.1" 200 -
^C
